In [25]:
from pydantic import BaseModel
from llama_index.core import (
    StorageContext,
    load_index_from_storage,
)
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.core.indices.vector_store.base import VectorStoreIndex
# from llama_index.core.indices.base import BaseIndex

from llama_index.core.base.embeddings.base import BaseEmbedding
from llama_index.core import Settings
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine


class Retriever():
    
    def __init__(self, index: VectorStoreIndex, embed_model: BaseEmbedding, reranker: FlagEmbeddingReranker=None, query_engine: RetrieverQueryEngine=None):
        self.index = index
        self.embed_model = embed_model
        self.reranker = reranker
        self.query_engine = query_engine

    @classmethod
    def load_index_from_disk(cls, PERSIST_DIR, embed_model_name="BAAI/bge-m3", pooling="mean"):
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
        embed_model = HuggingFaceEmbedding(
                        model_name=embed_model_name,
                        pooling=pooling # "cls" is default
                    )

        index = load_index_from_storage(storage_context, embed_model=embed_model, llm=None)
        return cls(index=index, embed_model=embed_model)

    def load_reranker(self, top_n=3, reranker_id="Dongjin-kr/ko-reranker"):
        self.reranker = FlagEmbeddingReranker(
                    top_n=top_n,
                    model=reranker_id,
                )

    def load_retriever_query_engine(self, top_k):
        Settings.llm = None
        retriever_query_engine = self.index.as_query_engine(similarity_top_k=top_k, node_postprocessors=[self.reranker], verbose=True)
        self.query_engine = retriever_query_engine
    

    
    def get_k_relevant_documents_with_reranker(self, query):
        relevent_docs = self.query_engine.query(query)
        return relevent_docs        

        
    def get_context_from_relevant_documents(self, relevent_docs):
        context_list = []
        for r in relevent_docs.source_nodes:
            if r.score > 0:
                context_list.append(r.text)
        return context_list


In [26]:

retriever = Retriever.load_index_from_disk(PERSIST_DIR="../index/train-vector-index-storage-chunk-size-1295", embed_model_name="BAAI/bge-m3")

In [27]:
retriever

In [28]:
retriever.load_reranker(top_n=3, reranker_id="Dongjin-kr/ko-reranker")

In [29]:
retriever.load_retriever_query_engine(top_k=2)

LLM is explicitly disabled. Using MockLLM.


In [30]:
vars(retriever)

{'index': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x72d682312320>,
 'embed_model': HuggingFaceEmbedding(model_name='BAAI/bge-m3', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x72d6ab973040>, tokenizer_name='BAAI/bge-m3', max_length=8194, pooling=<Pooling.MEAN: 'mean'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None),
 'reranker': FlagEmbeddingReranker(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x72d6ab973040>, model='Dongjin-kr/ko-reranker', top_n=3, use_fp16=False),
 'query_engine': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x72d682310220>}

In [31]:
relevant_docs = retriever.get_k_relevant_documents_with_reranker(query = "면진장치가 뭐야?")
context_list = retriever.get_context_from_relevant_documents(relevant_docs)
for c in context_list:
    print(c)

TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.


In [33]:
from pydantic import BaseModel

class Augment(BaseModel):
    prompt: str

    @classmethod
    def get_prompt(cls, retriever, question, prev_q=""):
        # prev_q is a must needed for some questions like this: What is the biggest cause of plaster revision? And please tell me how to solve this.”
        INSTRUCTION_PROMPT_TEMPLATE = """\
        ### System:
        벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.

        ### User:
        <보고서>
        {CONTEXT}
        </보고서>
        지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.
        <질문>
        {QUESTION}
        </질문>

        ### Assistant:
        """

        response_1 = retriever.query_engine.query(question)

        context_list = []
        for r in response_1.source_nodes:
            # print(r.score)
            if r.score > 0:
                if r.score <= 4 and len(context_list) >= 1:
                    pass
                else:
                    context_list.append(r.text)

        # Special case when the follow up question is junk
        if len(context_list) == 0:
            response_2 = retriever.query_engine.query(prev_q + " " + question)
            for r in response_2.source_nodes:
                if r.score > 0:
                    context_list.append(r.text)

        context = prev_q + "\n\n".join(context_list + [question])

        prompt = INSTRUCTION_PROMPT_TEMPLATE.format(CONTEXT=context, QUESTION=question)   

        return cls(prompt=prompt)     

In [34]:
question = "면진장치가 뭐야?"
augment = Augment.get_prompt(retriever, question)


In [36]:
augment.prompt

'        ### System:\n        벽지에 대한 고객 문의에 정확하고 유용한 답변을 작성한다. <질문>의 의도를 파악하여 정확하게 <보고서>만을 기반으로 답변하세요.\n\n        ### User:\n        <보고서>\n        TRAIN_000, 면진장치가 뭐야?, 면진장치에 사용되는 주요 기술은 무엇인가요?, 건축구조, 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다., 면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을 줄여주는 장치입니다. 주로 지진이나 기타 지반의 진동으로 인한 피해를 방지하기 위해 사용됩니다., 면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물의 안전성과 안정성을 향상시키고, 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지진으로 인한 건물의 피해를 최소화하기 위해 주로 사용됩니다., 면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 최소화해 주는 진동 격리장치입니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다., 면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다. 지반으로부터 오는 진동 에너지의 영향을 완화시키기 위해 사용됩니다.\n\n면진장치가 뭐야?\n        </보고서>\n        지침사항을 반드시 지키고, <보고서>를 기반으로 <질문>에 답변하세요.\n        <질문>\n        면진장치가 뭐야?\n        </질문>\n\n        ### Assistant:\n        '